In [45]:
import os
import requests
import base64
from datetime import datetime, timedelta

In [8]:
auth_url = "https://accounts.spotify.com/api/token"
client_id = os.getenv("SPOTIPY_CLIENT_ID")
client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")
redirect_uri = os.getenv("SPOTIPY_REDIRECT_URI")
base_url = "https://api.spotify.com/v1/" # base URL of all Spotify API endpoints

In [49]:
class SpotifyAPI():
    access_token = None
    access_token_expiry = datetime.now()
    client_id = None
    client_secret = None
    auth_url = 'https://accounts.spotify.com/api/token'
    
    
    def __init__(self, client_id, client_secret):
        '''
        Initialises the Spotify client with the client ID and secret
        '''
        self.client_id = client_id
        self.client_secret = client_secret
    
    
    def get_token_header(self):
        '''
        Returns the token header in base64 encoding necessary for the authorisation post request
        '''
        client_credentials_b64 = self.get_client_credentials()
        return {"Authorization": "Basic {credentials}".format(credentials=client_credentials_b64)}
    
    
    def get_token_data(self):
        '''
        Returns the token data necessary for the authorisation post request
        '''
        return {"grant_type": "client_credentials"}
    
    
    def get_client_credentials(self):
        ''' 
        Returns a base 64 encoded authorisation string
        '''
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id is None or client_secret is None:
            raise Exception("You must set a client ID and a client secret")
        client_credentials = f'{client_id}:{client_secret}'
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        return client_credentials_b64.decode()
    
    
    def authorise(self):
        '''
        Authorises the client, setting the authorisation token and returning True if successful
        '''
        auth_url = self.auth_url
        auth_data = self.get_token_data()
        auth_headers = self.get_token_header()
        auth_response = requests.post(auth_url, data=auth_data, headers=auth_headers)
        if auth_response.status_code not in range(200,299):
            return False
        auth_response_data = auth_response.json()
        access_token = auth_response_data["access_token"]
        self.access_token = access_token
        now = datetime.now()
        expires_in = auth_response_data['expires_in']
        expires = now + timedelta(seconds=expires_in)
        self.access_token_expires = expires
        self.access_token_expired = expires < now
        return True

In [50]:
spotify = SpotifyAPI(client_id, client_secret)

In [51]:
spotify.authorise()

True

In [52]:
spotify.access_token

'BQDnC1hhxRZcbJE1MQDcOLijCDbDh3wzhb_EU-lxUMLRbmlQYFTgJOYx3x7XUATxbOELhdKckUMYqdmtFDs'